# Development script for Lambert problem solver

Formulation follows Curtis Chapter 5.3 Lambert's Problem pg.247-

Python implementation by Yuri Shimane

In [8]:
import numpy as np
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt

In [9]:
from numpy import linalg as LA  # linear algebra functions in numpy

# define functions that will be used repeatedly in iterative solving step
def _Stumpff_S(z):
    """
    Stumpff function S(z)
    Args:
        z (float): universal anomaly^2/semi-major axis of transfer trajectory
    Returns:
        (float): value of Stumpff functio S(z) evaluated for input z
    """
    if z > 0:
        S = (np.sqrt(z) - np.sin(np.sqrt(z)))/np.power(z,1.5)
    elif z == 0:
        S = 1/6
    elif z < 0:
        S = (np.sinh(np.sqrt(-z)) - np.sqrt(-z))/np.power(-z,1.5)

    return S


def _Stumpff_C(z):
    """
    Stumpff function C(z)
    Args:
        z (float): universal anomaly^2/semi-major axis of transfer trajectory
    Returns:
        (float): value of Stumpff functio S(z) evaluated for input z
    """
    if z > 0:
        C = (1 - np.cos(np.sqrt(z)))/z
    elif z == 0:
        C = 1/2
    elif z < 0:
        C = (np.cosh(np.sqrt(-z)) - 1)/(-z)

    return C

def _y_538(r1,r2,A,z):
    """
    Intermediate function in Lambert problem derivation (eq.5.38 in Curtis)
    Args:
        r1 (1x3 numpy array): position vector of departure
        r2 (1x3 numpy array): position vector of arrival
        A (float): intermediate value related only to input parameters
        z (float): universal anomaly^2/semi-major axis of transfer trajectory
    Returns:
        (float): value of function evaluated for input z
    """

    y = LA.norm(r1) + LA.norm(r2) + A*(z*_Stumpff_S(z) - 1)/np.sqrt(_Stumpff_C(z))

    return y

In [53]:
# Lambert solver
def lambert(r1, r2, tof, mu, grade='pro', method=None, **kwargs):
    """
    Function takes in classic parameters to Lambert problem to determine orbitalelements
    Args:
        r1 (1x3 numpy array): initial position vector of departure [km]
        r2 (1x3 numpy array): final position vector of arrival [km]
        tof (float): time of flight [s]
        mu (float): gravitational parameter [km^3/s^2]
        grade (str): trajectory orientation ('pro' for prograde or 'retro' for retrograde)
        **kwargs: other arguments for scipy.optimize.root_scalar
    Returns:
        (tuple): velocity vector at position 1 and 2 of solution trajectory to Lambert problem
    """

    # compute dtheta [rad]
    dtheta = np.arccos( np.dot(r1,r2)/(LA.norm(r1)*LA.norm(r2)) )
    c12 = np.cross(r1,r2)
    
    print(dtheta)
    # update dtheta for special cases
    if grade=='retro':
        if c12[2] >= 0:
            dtheta = 2*np.pi - dtheta
    else:
        if c12[2] <= 0:
            dtheta = 2*np.pi - dtheta
    
    # retrgrade orbit
    #if grade=='retro':
    #    if tmp[1] < 0:
    #        dtheta = np.arccos(np.dot(r1,r2)/(LA.norm(r1)*LA.norm(r2)))
    #    else: 
    #        dtheta = 2*np.pi - np.arccos(np.dot(r1,r2)/(LA.norm(r1)*LA.norm(r2)))
    # prograde orbit
    #else:
    #    if tmp[1] < 0:
    #        dtheta = 2*np.pi - np.arccos(np.dot(r1,r2)/(LA.norm(r1)*LA.norm(r2)))
    #    else:
    #        dtheta = np.arccos(np.dot(r1,r2)/(LA.norm(r1)*LA.norm(r2)))
    
    print('dtheta: {}'.format(dtheta*180/(np.pi)))
    
    
    # compute input parameter A where A = sin(dtheta) * sqrt[r1*r2 / (1 - cos(dtheta))]
    A = np.sin(dtheta) * np.sqrt(LA.norm(r1)*LA.norm(r2)/(1 - np.cos(dtheta)))
    print(f'Value of A: {A}')
    
    
    # FIXME - if orbit is retrograde, override bracket_window to only have positive z-values
    bracket_window = (0.1,5000)
    z0 = -10
    
    # Scipy - NR method scipy.optimize.rootscalar
    def residue_Fz(z,r1,r2,A):
        """Function computes residue of F(z) as defined by Curtis eq. (5.40)
        Args:
            z 
        Returns:
            (tuple): tuple containing residue of F computed at z and Fdot eq. (5.43)
        """
        residue = np.power(_y_538(r1,r2,A,z)/_Stumpff_C(z), 3/2) * _Stumpff_S(z) + A*np.sqrt(_y_538(r1,r2,A,z)) - np.sqrt(mu)*tof
        
        if z == 0:
            Fdot = np.sqrt(2) * np.power(_y_538(r1,r2,A,0),1.5)/40 + (A/8)*(np.sqrt(_y_538(r1,r2,A,0)) + A*np.sqrt(1/(2*_y_538(r1,r2,A,0))))
        else:
            Fdot = np.power(_y_538(r1,r2,A,z)/_Stumpff_C(z), 1.5) * (((1/(2*z)) * (_Stumpff_C(z) - 3*_Stumpff_S(z)/(2*_Stumpff_C(z)))) + 3*np.power(_Stumpff_S(z),2)/(4*_Stumpff_C(z))) + (A/8)*(3*_Stumpff_S(z)*np.sqrt(_y_538(r1,r2,A,z))/_Stumpff_C(z) + A*np.sqrt(_Stumpff_C(z)/_y_538(r1,r2,A,z)))
    
        return residue, Fdot
    
    # solve to find z-value
    sol = opt.root_scalar(residue_Fz, args=(r1,r2,A), fprime=True, bracket=bracket_window, method=method, **kwargs)
    #sol = opt.root_scalar(residue_Fz, args=(r1,r2,A), fprime=True, x0=z0, method='method', **kwargs)
    
    if sol.converged:
        z1 = sol.root
    else:
        raise RuntimeError(f'F(z) = 0 calculation failed with initial guess of z {z0}')  # FIXME - document failure
    

    # display orbit type
    if z1 > 0:
        print(f'Transfer trajectory is an ellipse; z = {z1}')
    elif z1 == 0:
        print(f'Transfer trajectory is a parabola; z = {z1}')
    elif z1 < 0:
        print(f'Transfer trajectory is a hyperbolla; z = {z1}')

    # calculate Lagrange functions
    f    = 1 - _y_538(r1,r2,A,z1)/LA.norm(r1)
    g    = A*np.sqrt(_y_538(r1,r2,A,z1)/mu)
    gdot = 1 - _y_538(r1,r2,A,z1)/LA.norm(r2)
    print(f'Lagrange functions f: {f}, g: {g}, gdot: {gdot}')

    # calculate initial and final velocity vectors
    v1 = (1/g)*(r2 - f*r1)
    v2 = (1/g)*(gdot*r2 - r1)
    print('Velocity at r1: {} \nvelocity at r2: {}'.format(v1,v2))

    return v1, v2


In [54]:
# initial and final position vectors [km]
r_in = np.array([5000, 10000, 2100])
r_fn = np.array([-14600, 2500, 7000])

# time of flight [s]
dt = 3600

# gravitational parameter [km^3/s^2]
mu_E = 398600

# solve Lambert problem
#v1, v2 = lb.lambert(r1=r_in, r2=r_fn, tof=dt, mu=mu_E, grade='pro')
# obtain orbital elements
#elements = oc.sv2el(r_in,v1,mu_E)

In [55]:
# solve Lamert problem in prograde
v1, v2 = lambert(r1=r_in, r2=r_fn, tof=dt, mu=mu_E, grade='pro')
# FIXME: z value is wrong!

1.7504347625534349
dtheta: 100.29252420729621
Value of A: 12372.272033956451
Transfer trajectory is an ellipse; z = 1.5398544345070695
Lagrange functions f: -0.18876687819739923, g: 2278.8782352207904, gdot: 0.1745680621662019
Velocity at r1: [-5.99249464  1.92536342  3.24563653] 
velocity at r2: [-3.31246031 -4.19661731 -0.38528762]


In [59]:
# solve Lambert problem in prograde with more extreme tof
dt2 = 2000
v1, v2 = lambert(r1=r_in, r2=r_fn, tof=dt2, mu=mu_E, grade='pro')


1.7504347625534349
dtheta: 100.29252420729621
Value of A: 12372.272033956451


ValueError: f(a) and f(b) must have different signs